In [1]:
# import tensorflow as tf
# from tensorflow.python.client import device_lib
# tf.config.list_physical_devices('GPU')

In [2]:
# device_lib.list_local_devices()

In [3]:
# pwd

In [1]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer,util
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
stop_words=stopwords.words('english')
punctuation=string.punctuation

C:\ProgramData\Anaconda3\envs\nls_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_custom_embeddings(model_path,corpus_embeddings):
    #store sentences & embeddings on disc
    with open(model_path+'\\'+'embeddings.pkl',"wb") as fout:
        pickle.dump({'Sentences':corpus, 'embeddings': corpus_embeddings},fout)
    print("saved Custom embeddings")

def load_custom_embeddings(model_path):
    with open(model_path+'/embeddings.pkl',"rb") as fin:
        stored_data = pickle.load(fin)
        stored_sentences = stored_data['Sentences']
        stored_embeddings = stored_data['embeddings']
    return stored_sentences,stored_embeddings

def get_embeddings(sentence):
    #encode sentence to get sentence embeddings
    sentence_embedding=model.encode(sentence, convert_to_tensor=True)
    return sentence_embedding

def sentence_similarity_scores(sentence_embedding,
                              custom_embeddings,
                              stored_sentences,
                              top_k,
                              input_sentence):
    #computing similarity scores with the corpus
    cos_scores= util.pytorch_cos_sim(sentence_embedding, custom_embeddings)[0]
    #sort the results in decreasing order and get the first top_k
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    print("sentence :", input_sentence, "\n")
    print("Top", top_k, "most similar sentences in corpus")
    results={}
    for idx in top_results[0:top_k]:
        print(stored_sentences[idx],"(scores:%4f)" % (cos_scores[idx]))
        results[f"sentence{idx}"]= ({"predicted_sentence": stored_sentences[idx],"Scores" : float(cos_scores[idx])})
    return results

def clean_text(text):
    # Convert the text to title case
    text = str(text).title()
    # Remove the punctuation
    text = ''.join([c for c in text if c not in punctuation])
    # Remove the stop words
    tokens = [token for token in text.split() if token.lower() not in stop_words]
    # Convert the tokens back to a string
    cleaned_text = ' '.join(tokens)
    return cleaned_text


def concate_column_text(data):
    df["concated_text"]=df[["Category Name","Service name","Service Classification"]].astype(str).agg(' '.join,axis=1)
    return df["concated_text"]


def convert_column_to_list(data):
    data=data.tolist()
    return data

def convert_df_to_list(data):
    all_data=[]
    corpus=[]
    for values in df.columns:
        listin=df[values].tolist()
        all_data.append(listin)
    complete_data = [element for innerList in all_data for element in innerList]
    for word in complete_data:
        if word not in corpus:
            corpus.append(word)
    return corpus

In [3]:
# Define the path to the Excel files
path_to_files = r"C:\Users\AL44096\Documents\NLS_excel_files"

# Define the names of the Excel files
file_names = ['Benefit_description_short.xlsx'
              ]

# Define the name of the sheet in each Excel file that contains the text data
#sheet_name = 'Sheet1'

# Load and clean the text data from each Excel file
cleaned_data = []
for file_name in file_names:
    # Load the data from the Excel file into a Pandas DataFrame
    df = pd.read_excel(f'{path_to_files}/{file_name}')
    #data=concate_column_text(df)
    data=df['Description']
    #cleaned_data.append(data)
    # Extract the relevant text data from the DataFrame
    text_data = convert_column_to_list(data)
    # Clean the text data
    cleaned_text_data = [clean_text(text) for text in text_data]
    cleaned_data.append(cleaned_text_data)
corpus = [element for innerList in cleaned_data for element in innerList]

In [4]:
corpus

['Federal Law Prohibits Federal Funds Including Marketplace Premium Tax Credits Used Abortion Services Except Limited Cases Rape Incest Woman Suffers LifeThreatening Physical Injury Illness Would Place Woman Danger Death Unless Abortion Performed Marketplace Health Plans May Cover Abortion Services Different Ways Abortion Coverage Restrictions Abortion Coverage Services Cannot Paid Federal Dollars Known “NonHyde” Abortion Services Abortion Coverage Contact Plan Provider Learn Coverage Abortion Services Comparing Plans HealthcareGov Plan Details Screen Show Plan Offers Abortion Coverage Federal Dollars Cannot Used',
 'Group Health Care Providers Give Coordinated Care Chronic Disease Management Thereby Improve Quality Care Patients Get OrganizationS Payment Tied Achieving Health Care Quality Goals Outcomes Result Cost Savings',
 'Marketplace Health Plan Approved “Seal Approval” Given Plan Independent Organization Show Plan Meets National Quality Standards',
 'Percentage Total Average Cos

In [5]:
#len(df['concated_text'])
df=df.groupby(['Benefit_name','Web_link'])['Description'].sum()

In [6]:
#df = df.drop(columns=['Unnamed: 0'])
df=df.to_frame()
df

,,Description
Benefit_name,Web_link,
Abortion services,https://www.healthcare.gov/glossary/abortion-services/,"Federal law prohibits federal funds, including..."
Accountable care organization,https://www.healthcare.gov/glossary/accountable-care-organization/,A group of health care providers who give coor...
Accreditation,https://www.healthcare.gov/glossary/accreditation/,"If a Marketplace health plan is approved, this..."
Actuarial value,https://www.healthcare.gov/glossary/actuarial-value/,The percentage of total average costs for cove...
Adjusted Gross Income (AGI),https://www.healthcare.gov/glossary/adjusted-gross-income-agi/,Your total (or “gross”) income for the tax yea...
...,...,...
Waiting period (job-based coverage),https://www.healthcare.gov/glossary/waiting-period-job-based-coverage/,The time that must pass before coverage can be...
Well-baby and well-child visits,https://www.healthcare.gov/glossary/well-baby-and-well-child-visits/,Routine doctor visits for comprehensive preven...
Wellness program,https://www.healthcare.gov/glossary/wellness-programs/,A program intended to improve and promote heal...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 257 entries, ('Abortion services', 'https://www.healthcare.gov/glossary/abortion-services/') to ('Zero cost sharing plan', 'https://www.healthcare.gov/glossary/zero-cost-sharing-plan/')
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  257 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


In [8]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['Description', 'Benefit_name', 'Web_link'],
    num_rows: 257
})

In [12]:
#comments_dataset['concated_text'][0]

In [13]:
# from sentence_transformers import SentenceTransformer
sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding_dim = 384

#Define input_layer
input_layer = Input(shape=(None,embedding_dim)), name="input_layer")

In [16]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [17]:
# embedding = get_embeddings(comments_dataset["concated_text"][0])
# embedding.shape

In [18]:
comments_dataset

Dataset({
    features: ['Description', 'Benefit_name', 'Web_link'],
    num_rows: 257
})

In [19]:
import numpy
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["Description"]).numpy()}
)

In [20]:
embeddings_dataset

Dataset({
    features: ['Description', 'Benefit_name', 'Web_link', 'embeddings'],
    num_rows: 257
})

In [ ]:
#df2=pd.DataFrame(embeddings_dataset)

In [ ]:
#df2.head()

In [21]:
len(embeddings_dataset['embeddings'])

257

In [22]:
embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.46it/s]


Dataset({
    features: ['Description', 'Benefit_name', 'Web_link', 'embeddings'],
    num_rows: 257
})

In [23]:
question = "WheelChair"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 384)

In [24]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

In [25]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [26]:
for _, row in samples_df.iterrows():
    #print(row)
    print(f"Service name: {row['Benefit_name']}")
    print(f"SCORE: {row.scores}")
    print('\n')
    print(f"concated_text: {row.Description}")
    print('\n')

Service name: Uncompensated care
SCORE: 63.21757888793945


concated_text: Health care or services provided by hospitals or health care providers that don't get reimbursed. Often uncompensated care arises when people don't have insurance and cannot afford to pay the cost of care.


Service name: Physician services
SCORE: 63.10593032836914


concated_text: Health care services a licensed medical physician (M.D. – Medical Doctor or D.O. – Doctor of Osteopathic Medicine) provides or coordinates.


Service name: Reconstructive surgery
SCORE: 62.73225021362305


concated_text: Surgery and follow-up treatment needed to correct or improve a part of the body because of birth defects, accidents, injuries or medical conditions.


Service name: Medicare
SCORE: 62.71996307373047


concated_text: A federal health insurance program for people 65 and older and certain younger people with disabilities. It also covers people with End-Stage Renal Disease (permanent kidney failure requiring dialysis or a

In [ ]:
#str(corpus_embeddings)

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [29]:
corpus_embeddings =  model.encode(corpus, convert_to_tensor=True)

In [30]:
corpus_embeddings

tensor([[ 0.0187,  0.0305, -0.0664,  ..., -0.0724,  0.1611,  0.0365],
        [-0.0351,  0.0541,  0.0044,  ..., -0.0526,  0.0449, -0.0026],
        [-0.0202,  0.0573,  0.0462,  ..., -0.1122,  0.0941,  0.0704],
        ...,
        [-0.0315,  0.0818,  0.0674,  ..., -0.0439,  0.0765, -0.0317],
        [-0.0228,  0.1304,  0.0545,  ..., -0.1139,  0.0366,  0.1009],
        [-0.0866,  0.0267, -0.0048,  ..., -0.0461, -0.0224,  0.0615]])

In [31]:
import pickle
model_path= r"C:\Users\AL44096\Documents\sentence_model"
save_custom_embeddings(model_path,corpus_embeddings)

saved Custom embeddings


In [32]:
stored_sentences,stored_embeddings=load_custom_embeddings(model_path)

In [33]:
stored_embeddings

tensor([[ 0.0187,  0.0305, -0.0664,  ..., -0.0724,  0.1611,  0.0365],
        [-0.0351,  0.0541,  0.0044,  ..., -0.0526,  0.0449, -0.0026],
        [-0.0202,  0.0573,  0.0462,  ..., -0.1122,  0.0941,  0.0704],
        ...,
        [-0.0315,  0.0818,  0.0674,  ..., -0.0439,  0.0765, -0.0317],
        [-0.0228,  0.1304,  0.0545,  ..., -0.1139,  0.0366,  0.1009],
        [-0.0866,  0.0267, -0.0048,  ..., -0.0461, -0.0224,  0.0615]])

# searching through faiss index

In [34]:
d = stored_embeddings.shape[1]

In [35]:
import faiss
index = faiss.IndexFlatL2(d)

In [36]:
index.is_trained

True

In [37]:
index.add(stored_embeddings)

In [38]:
index.ntotal

257

In [39]:
k = 10
xq = model.encode(["WheelChair"])

In [40]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 64 113 207 106 249  63 245 172 139 194]]
CPU times: total: 0 ns
Wall time: 0 ns


In [41]:
[f'{i}: {corpus[i]}' for i in I[0]]

['64: Ambulance Services Emergency Medical Condition',
 '113: Services Provide Comfort Support Persons Last Stages Terminal Illness Families',
 '207: Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings',
 '106: Refers Medical Conditions Physical Mental Health Claims Experience Receipt Health Care Medical History Genetic Information Evidence Insurability Disability',
 '249: Care Illness Injury Condition Serious Enough Reasonable Person Would Seek Care Right Away Severe Requires Emergency Room Care',
 '63: Illness Injury Symptom Condition Serious Reasonable Person Would Seek Care Right Away Avoid Severe Harm',
 '245: Tty Teletypewriter Communication Device Used People Deaf HardOfHearing Severe Speech Impairment People DonT Tty Communicate Tty User Message Relay Center Mrc Mr

In [42]:
similar_sentences=[corpus[idx] for idx in I[0]]
similar_probabilities=[1-distance for distance in D[0]]

In [43]:
for sentence, probability in zip(similar_sentences,similar_probabilities):
    print("Similar Sentence :", sentence)
    print("probability :", probability)
    print()

Similar Sentence : Ambulance Services Emergency Medical Condition
probability : -0.35128748416900635

Similar Sentence : Services Provide Comfort Support Persons Last Stages Terminal Illness Families
probability : -0.44650161266326904

Similar Sentence : Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings
probability : -0.4689595699310303

Similar Sentence : Refers Medical Conditions Physical Mental Health Claims Experience Receipt Health Care Medical History Genetic Information Evidence Insurability Disability
probability : -0.47084474563598633

Similar Sentence : Care Illness Injury Condition Serious Enough Reasonable Person Would Seek Care Right Away Severe Requires Emergency Room Care
probability : -0.4766143560409546

Similar Sentence : Illness Injury Symptom Conditio

# searching by creating an clusters

In [44]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [45]:
index.is_trained

False

In [46]:
index.train(stored_embeddings)
index.is_trained # check if index is now trained

True

In [47]:
index.add(stored_embeddings)
index.ntotal  # number of embeddings indexed

257

In [48]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[207 245 206 222 247 124  96 133 169  40]]
CPU times: total: 15.6 ms
Wall time: 998 µs


In [49]:
[f'{i}: {corpus[i]}' for i in I[0]]

['207: Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings',
 '245: Tty Teletypewriter Communication Device Used People Deaf HardOfHearing Severe Speech Impairment People DonT Tty Communicate Tty User Message Relay Center Mrc Mrc Tty Operators Available Send Interpret Tty Messages',
 '206: Written Order Primary Care Doctor See Specialist Get Certain Medical Services Many Health Maintenance Organizations Hmos Need Get Referral Get Medical Care Anyone Except Primary Care Doctor Don’T Get Referral First Plan May Pay Services',
 '222: Skilled Nursing Care Rehabilitation Services Provided Continuous Daily Basis Skilled Nursing Facility Examples Skilled Nursing Facility Care Include Physical Therapy Intravenous Injections Given Registered Nurse Doctor',
 '247: Health Care Servic

In [50]:
similar_sentences=[corpus[idx] for idx in I[0]]
similar_probabilities=[1-distance for distance in D[0]]

In [51]:
for sentence, probability in zip(similar_sentences,similar_probabilities):
    print("Similar Sentence :", sentence)
    print("probability :", probability)
    print()

Similar Sentence : Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings
probability : -0.4689595699310303

Similar Sentence : Tty Teletypewriter Communication Device Used People Deaf HardOfHearing Severe Speech Impairment People DonT Tty Communicate Tty User Message Relay Center Mrc Mrc Tty Operators Available Send Interpret Tty Messages
probability : -0.4884023666381836

Similar Sentence : Written Order Primary Care Doctor See Specialist Get Certain Medical Services Many Health Maintenance Organizations Hmos Need Get Referral Get Medical Care Anyone Except Primary Care Doctor Don’T Get Referral First Plan May Pay Services
probability : -0.5699763298034668

Similar Sentence : Skilled Nursing Care Rehabilitation Services Provided Continuous Daily Basis Skilled Nursing Facili

# Searching by all the 10 nearest clusters

In [52]:
index.nprobe = 10

In [53]:
%%time
D, I = index.search(xq, k)  # search
print(I)
print(D)

[[ 64 113 207 106 249  63 245 172 194 195]]
[[1.3512875 1.4465016 1.4689596 1.4708447 1.4766144 1.4881022 1.4884024
  1.5076123 1.5297649 1.5434499]]
CPU times: total: 0 ns
Wall time: 0 ns


In [54]:
similar_sentences=[corpus[idx] for idx in I[0]]
similar_probabilities=[1-distance for distance in D[0]]

In [55]:
for sentence, probability in zip(similar_sentences,similar_probabilities):
    print("Similar Sentence :", sentence)
    print("probability :", probability)
    print()

Similar Sentence : Ambulance Services Emergency Medical Condition
probability : -0.35128748416900635

Similar Sentence : Services Provide Comfort Support Persons Last Stages Terminal Illness Families
probability : -0.44650161266326904

Similar Sentence : Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings
probability : -0.4689595699310303

Similar Sentence : Refers Medical Conditions Physical Mental Health Claims Experience Receipt Health Care Medical History Genetic Information Evidence Insurability Disability
probability : -0.47084474563598633

Similar Sentence : Care Illness Injury Condition Serious Enough Reasonable Person Would Seek Care Right Away Severe Requires Emergency Room Care
probability : -0.4766143560409546

Similar Sentence : Illness Injury Symptom Conditio

In [56]:
[f'{i}: {corpus[i]}' for i in I[0]]

['64: Ambulance Services Emergency Medical Condition',
 '113: Services Provide Comfort Support Persons Last Stages Terminal Illness Families',
 '207: Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings',
 '106: Refers Medical Conditions Physical Mental Health Claims Experience Receipt Health Care Medical History Genetic Information Evidence Insurability Disability',
 '249: Care Illness Injury Condition Serious Enough Reasonable Person Would Seek Care Right Away Severe Requires Emergency Room Care',
 '63: Illness Injury Symptom Condition Serious Reasonable Person Would Seek Care Right Away Avoid Severe Harm',
 '245: Tty Teletypewriter Communication Device Used People Deaf HardOfHearing Severe Speech Impairment People DonT Tty Communicate Tty User Message Relay Center Mrc Mr

In [57]:
p=[corpus[i] for i in I[0]]
p

['Ambulance Services Emergency Medical Condition',
 'Services Provide Comfort Support Persons Last Stages Terminal Illness Families',
 'Health Care Services Help Keep Get Back Improve Skills Functioning Daily Living Lost Impaired Sick Hurt Disabled Services May Include Physical Occupational Therapy SpeechLanguage Pathology Psychiatric Rehabilitation Services Variety Inpatient AndOr Outpatient Settings',
 'Refers Medical Conditions Physical Mental Health Claims Experience Receipt Health Care Medical History Genetic Information Evidence Insurability Disability',
 'Care Illness Injury Condition Serious Enough Reasonable Person Would Seek Care Right Away Severe Requires Emergency Room Care',
 'Illness Injury Symptom Condition Serious Reasonable Person Would Seek Care Right Away Avoid Severe Harm',
 'Tty Teletypewriter Communication Device Used People Deaf HardOfHearing Severe Speech Impairment People DonT Tty Communicate Tty User Message Relay Center Mrc Mrc Tty Operators Available Send In

In [ ]:
for i in p:
    y=df.loc[df["concated_text"]==i]['Service name']#.values[0]#item()#iloc[0]
    print(y)#.to_string())

In [ ]:
df.iloc[11394]['Service name']